<a href="https://colab.research.google.com/github/talpt/pyton/blob/main/Kompozit_Echo_Trend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/rongardF/tvdatafeed  backtesting
!pip install tradingview-screener==2.5.0
!pip install openpyxl
!pip install statsmodels
!pip install tqdm
!pip install tradingview-ta

import pandas as pd
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols

# EMA hesaplama fonksiyonu
def ema(src, span):
    return src.ewm(span=span, adjust=False).mean()

# Normal RSI hesaplama fonksiyonu (RMA kullanarak)
def calculate_rsi(data, period=14):
    delta = data.diff()  # Fiyat değişimini hesapla
    gain = delta.where(delta > 0, 0)  # Kazançları filtrele
    loss = -delta.where(delta < 0, 0)  # Kayıpları filtrele

    avg_gain = gain.rolling(window=period, min_periods=1).mean()  # Kazançların ortalaması
    avg_loss = loss.rolling(window=period, min_periods=1).mean()  # Kayıpların ortalaması

    rs = avg_gain / avg_loss  # Kazanç kayıp oranı
    rsi = 100 - (100 / (1 + rs))  # RSI hesaplama
    return rsi


# Göreceli RSI hesaplama fonksiyonu
def calculate_relative_rsi(data, rsi_period=14, relative_rsi_period=14):
    # Normal RSI hesapla
    rsi = calculate_rsi(data, rsi_period)

    # RSI'nın hareketli ortalamasını al
    relative_rsi = rsi.rolling(window=relative_rsi_period, min_periods=1).mean()
    return relative_rsi

# Zaman dilimi seçimi fonksiyonu
def select_interval():
    print("\nLütfen zaman dilimini seçin:")
    print("1: Günlük")
    print("2: Haftalık")
    print("3: Aylık")
    choice = input("Seçiminiz (1/2/3): ")

    interval_map = {
        "1": Interval.in_daily,    # Günlük
        "2": Interval.in_weekly,   # Haftalık
        "3": Interval.in_monthly   # Aylık
    }

    return interval_map.get(choice, Interval.in_daily)  # Varsayılan olarak günlük interval seçilsin

# TradingView bağlantısı
tv = TvDatafeed()

# Zaman dilimini kullanıcıdan alıyoruz
selected_interval = select_interval()

# Hisseler listesini otomatik çekmek için tradingview-screener kullanıyoruz
Hisseler = get_all_symbols(market='turkey')  # Türkiye borsasındaki tüm hisseleri al

# Sonuçları saklamak için DataFrame
columns = ['Hisse', 'Fiyat', 'Fiyat/XU100', 'Normal_RSI', 'Göreceli_RSI',
           'EMA233_Durumu', 'EMA144_Durumu', 'EMA89_Durumu', 'EMA34_Durumu', 'Üstünde']
df_results = pd.DataFrame(columns=columns)

# Toplam hisse sayısını öğreniyoruz
total_hisseler = len(Hisseler)

for index, hisse in enumerate(Hisseler, start=1):
    try:
        # Seçilen zaman dilimine göre hisse ve XU100 verilerini alma
        data = tv.get_hist(symbol=hisse, exchange='BIST', interval=selected_interval, n_bars=300)
        dataref = tv.get_hist(symbol='XU100', exchange='BIST', interval=selected_interval, n_bars=300)

        # Verileri resetliyoruz (index sıfırlama)
        data = data.reset_index()
        dataref = dataref.reset_index()

        # Fiyatları XU100 endeksine göre normalize etme
        result = data.copy()
        columns_to_divide = ['open', 'high', 'low', 'close', 'volume']
        for column in columns_to_divide:
            result[column] = 100 * (data[column] / dataref[column])

        # EMA hesaplama
        result['EMA_233'] = ema(result['close'], 233)
        result['EMA_144'] = ema(result['close'], 144)
        result['EMA_89'] = ema(result['close'], 89)
        result['EMA_34'] = ema(result['close'], 34)

        # Normal RSI hesaplama
        result['Normal_RSI'] = calculate_rsi(result['close'])

        # Göreceli RSI hesaplama
        result['Göreceli_RSI'] = calculate_relative_rsi(result['close'])

        # EMA durum kontrolü
        latest = result.iloc[-1]
        ema_durumları = {
            'EMA233_Durumu': 'Üstünde' if latest['close'] > latest['EMA_233'] else 'Altında',
            'EMA144_Durumu': 'Üstünde' if latest['close'] > latest['EMA_144'] else 'Altında',
            'EMA89_Durumu': 'Üstünde' if latest['close'] > latest['EMA_89'] else 'Altında',
            'EMA34_Durumu': 'Üstünde' if latest['close'] > latest['EMA_34'] else 'Altında'
        }

        # "Üstünde" toplam sayısı
        üstünde_sayısı = sum([1 for durum in ema_durumları.values() if durum == 'Üstünde'])

        # Satır ekleme
        row = {
            'Hisse': hisse,
            'Fiyat': round(latest['close'], 2),
            'Fiyat/XU100': round(latest['close'] / dataref['close'].iloc[-1], 6),  # XU100 ile normalize edilmiş fiyat
            'Normal_RSI': round(latest['Normal_RSI'], 2),
            'Göreceli_RSI': round(latest['Göreceli_RSI'], 2),  # Göreceli RSI
            **ema_durumları,
            'Üstünde': üstünde_sayısı
        }

        # Sonuçları DataFrame'e ekle
        df_results = pd.concat([df_results, pd.DataFrame([row])], ignore_index=True)

        # İlerlemeyi sayısal olarak göster
        print(f"İşlenen Hisse: {hisse} ({index}/{total_hisseler})")

    except Exception as e:
        print(f"{hisse} için veri alınamadı: {e}")

# Tabloyu Excel'e kaydetme
df_results.to_excel("hisse_analiz_tablosu.xlsx", index=False)
print("Tablo başarıyla kaydedildi: hisse_analiz_tablosu.xlsx")

# Tarama tamamlandığında tabloyu ekrana yazdırma
print("\nTarama tamamlandı! Sonuçlar:")
print(df_results)


  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-tb_66tcw
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-tb_66tcw
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tvdatafeed: filename=tvdatafeed-2.1.0-py3-none-any.whl size=17533 sha256=3d2436b37abb77d713541f420cbc241f8fae3872f787f8b3885507d4dcb37bee
  Stored in directory: /tmp/pip-ephem-wheel-cache-wap_96ld/wheels/e4/32/1e/21ebcacc6549d75fae3bf3ff75cee8fcbe4e5c189d88fbd5a1
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173912 sha256=23b6733891d32f8d6b0ac4c561492b0977742c50189be59b929d2b222a364483
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f51034324906


Lütfen zaman dilimini seçin:
1: Günlük
2: Haftalık
3: Aylık
Seçiminiz (1/2/3): 1


<ipython-input-2-298e11b8ce8b>:124: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_results = pd.concat([df_results, pd.DataFrame([row])], ignore_index=True)


İşlenen Hisse: BIST:SAFKR (1/584)
İşlenen Hisse: BIST:BOBET (2/584)
İşlenen Hisse: BIST:TLMAN (3/584)
İşlenen Hisse: BIST:ATLAS (4/584)
İşlenen Hisse: BIST:PAMEL (5/584)
İşlenen Hisse: BIST:CANTE (6/584)
İşlenen Hisse: BIST:DOGUB (7/584)
İşlenen Hisse: BIST:A1CAP (8/584)
İşlenen Hisse: BIST:MAALT (9/584)
İşlenen Hisse: BIST:TATEN (10/584)
İşlenen Hisse: BIST:ENKAI (11/584)
İşlenen Hisse: BIST:ISFIN (12/584)
İşlenen Hisse: BIST:PSDTC (13/584)
İşlenen Hisse: BIST:AEFES (14/584)
İşlenen Hisse: BIST:GOKNR (15/584)
İşlenen Hisse: BIST:KENT (16/584)
İşlenen Hisse: BIST:MRGYO (17/584)
İşlenen Hisse: BIST:ISGYO (18/584)
İşlenen Hisse: BIST:ADGYO (19/584)
İşlenen Hisse: BIST:UNLU (20/584)
İşlenen Hisse: BIST:DURKN (21/584)
İşlenen Hisse: BIST:Z30KE (22/584)
İşlenen Hisse: BIST:DOCO (23/584)
İşlenen Hisse: BIST:ASELS (24/584)
İşlenen Hisse: BIST:MARKA (25/584)
İşlenen Hisse: BIST:TUKAS (26/584)
İşlenen Hisse: BIST:AYGAZ (27/584)
İşlenen Hisse: BIST:MOGAN (28/584)
İşlenen Hisse: BIST:SUMAS (29/58